In [12]:
from googlesearch import search
from bs4 import BeautifulSoup
import concurrent.futures
from urllib.parse import urlparse, urljoin
import urllib.robotparser
import urllib.request
import queue

In [11]:
def google_search(query):
    results_list = []
    for result in search(query,        # The query you want to run
                    tld = 'com',  # The top level domain
                    lang = 'en',  # The language
                    num = 10,     # Number of results per page
                    start = 0,    # First result to retrieve
                    stop = 10,  # Last result to retrieve
                
                ):
        results_list.append(result)
    return results_list

In [6]:
def create_robot_parser(url):
    rp = urllib.robotparser.RobotFileParser()
    rp.set_url(url+"/robots.txt")
    rp.read()
    return rp

rp=create_robot_parser("https://engineering.nyu.edu/")
print(rp.can_fetch("*", "https://engineering.nyu.edu/academics/programs"))
print(rp.can_fetch("*", "https://engineering.nyu.edu/profiles/"))

True
False


In [14]:
def read_url(url):
    try:
        page=urllib.request.urlopen(url)
    except urllib.error.HTTPError as error:
        return None,url,error.code
    except urllib.error.URLError:
        return None,url,'invalid url'
    return page.read(),url,200

In [23]:
def crawl(url, index):
    try:
        page=urllib.request.urlopen(url)
    except urllib.error.HTTPError as error:
        print(index, url+" "+str(error.code))
        return None
    except urllib.error.URLError:
        print(index, url+" "+"URLError")
        return None
    content=page.read()
    size=len(content)/1024
    print(index, url+" Size: "+"{:.2f}".format(size)+"kb")
    return content,url,200
print(crawl("http://mit.edu",1))
print(crawl("https://www.dnb.com/business-directory/company-profiles.the_brooklyn_union_gas_company.89887e0b5ad7b8157fc9e5f233efd2a8.html",2))

1 http://mit.edu Size: 46.40kb
47518
2 https://www.dnb.com/business-directory/company-profiles.the_brooklyn_union_gas_company.89887e0b5ad7b8157fc9e5f233efd2a8.html 403
None


In [8]:
def get_links(page_content):
    if page_content is None:
        return []
    soup = BeautifulSoup(page_content, "html.parser")
    links = []
    base = soup.find_all('base')
    if base:
        url=base[0].get('href')
    for link in soup.find_all('a'):
        #filter the links
        #non-url
        #modify relative url to common url
        href=link.get('href')
        parse_result=urlparse(href)
        if href and href[0]!="/" and href[0]!="#":
            if href[:4] != "http":
                continue
        if parse_result.scheme=="":
            links.append(urljoin(url,href))
        else:
            links.append(href)
    return links
# get_links('http://nyu.edu')
# get_links("https://www.dnb.com/business-directory/company-profiles.the_brooklyn_union_gas_company.89887e0b5ad7b8157fc9e5f233efd2a8.html")

In [13]:
def bfs_crawler(query):
    seen=set()
    scheduled_read=[]
    crawled_site={}
    q=queue.Queue()
    robot_parsers={}
    counter=0
    results_list=google_search(query)
    batch=queue.Queue()
    results=[]
    for link in results_list:
        parse_result=urlparse(link)
        parsed_link=urljoin(parse_result.scheme+"://"+parse_result.netloc,parse_result.path)
        netloc=parse_result.netloc
        if parsed_link not in seen:
            if netloc not in crawled_site or crawled_site[netloc]<10:
                q.put(link)
                seen.add(link)
                if netloc not in crawled_site:
                    crawled_site[netloc]=1
                else:
                    crawled_site[netloc]+=1
    with concurrent.futures.ThreadPoolExecutor() as executor:
        while len(results)<50 and not q.empty():
            for i in range(10):
                cur_url=q.get()
                print("putting into batch",counter,cur_url)
                # print(crawl(cur))
                scheduled_read.append((counter,cur_url))
                counter+=1
            batch_results=[executor.submit(read_url,url) for url in scheduled_read]
            for job in concurrent.futures.as_completed(batch_results):
                read_result=job.result()
                print(read_result)
                batch.put(read_result)
                print(crawl())
            for 
            for link in get_links(cur_url):
                if link:
                    parse_result=urlparse(link)
                    # parsed_link=urljoin(parse_result.scheme+"://"+parse_result.netloc,parse_result.path)
                    netloc=parse_result.netloc
                    if link not in seen:
                        #up to 10 page from a netloc can be crawled
                        if netloc not in crawled_site or crawled_site[netloc]<10:
                            current_site=parse_result.scheme+"://"+parse_result.netloc
                            if current_site not in robot_parsers:
                                robot_parsers[current_site]=create_robot_parser(current_site)
                            robot_parser=robot_parsers[current_site]
                            if robot_parser.can_fetch("*",link):
                                q.put(link)
                                seen.add(link)
                                if netloc not in crawled_site:
                                    crawled_site[netloc]=1
                                else:
                                    crawled_site[netloc]+=1
    
        
bfs_crawler("brooklyn union")

crawling 0 https://www.nationalgridus.com/
crawling 1 https://www1.nationalgridus.com/signin
crawling 2 https://www.nationalgridus.com/contact-us
crawling 3 https://www.nationalgridus.com/NY-Home/
crawling 4 https://www.nationalgridus.com/regionalized/Billing-Payments/Ways-to-Pay
crawling 5 https://twitter.com/JibreelJalloh/status/1437892570831593474?ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Etweet
crawling 6 https://twitter.com/countyofunionnj/status/1438204347612020740?ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Etweet
crawling 7 https://twitter.com/kelsey_caine/status/1438182101564731398?ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Etweet
crawling 8 https://twitter.com/search?q=brooklyn+union&ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Esearch
crawling 9 https://www.dnb.com/business-directory/company-profiles.the_brooklyn_union_gas_company.89887e0b5ad7b8157fc9e5f233efd2a8.html
crawling 10 https://www1.nationalgridus.com/outages
crawling 11 https://www1.nationalgridus.co

In [10]:
#priority queue
#class ScoredUrl

novelty_res={}
candidate_urls={}
url_pos={}
class ScoredUrl:
    def __init__(self, url):
        self.url=url
        self.netloc=urlparse(url).netloc
        self.importance=0
        if self.netloc not in novelty_res:
            novelty_res[self.netloc]=10

    def __eq__(self,other):
        return novelty_res[self.netloc]+self.importance==novelty_res[other.netloc]+other.importance

    def __lt__(self,other):
        return novelty_res[self.netloc]+self.importance<novelty_res[other.netloc]+other.importance

    def __gt__(self,other):
        return self>other

    def add_importance(self):
        self.importance+=1

    def crawl(self):
        return get_links(self.url)



class UrlQueue:
    def __init__(self):
        self.res=[]
    
    def add(self,new_url):
        heapq.heappush(self.res,new_url)
    
    def update(self,new_url):
        while
    #update

SyntaxError: invalid syntax (Temp/ipykernel_21872/177025105.py, line 40)